In [1]:
from Pipeline import Pipeline
import Adapters
import Processors
import Sinks

05/19/2021 14:47:24 - INFO - faiss.loader -   Loading faiss with AVX2 support.
05/19/2021 14:47:24 - INFO - faiss.loader -   Loading faiss.
/home/daniel/anaconda3/envs/KD/lib/python3.8/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0
05/19/2021 14:47:25 - INFO - farm.modeling.prediction_head -   Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .


### Pipeline Definition

In [2]:
adapter = Adapters.UnarxiveAdapter("../../data/unarXive-sample")

In [3]:
pipeline = Pipeline(adapter, batch_size=20)

In [4]:
#MetadataArxiveEnricher
from arxive_metadata.rocksDB import RocksDBAdapter
db = RocksDBAdapter("arXive_metadata", "http://localhost:8089")
pipeline.add_processor(Processors.MetadataArxiveEnricher("arixive-id",db))

In [5]:
#FilterOnMetadataValue
cs_values = [
    "cs.AI", 
    "cs.AR", 
    "cs.CC", 
    "cs.CE", 
    "cs.CG", 
    "cs.CL", 
    "cs.CR",
    "cs.CV",
    "cs.CY",
    "cs.DB",
    "cs.DC",
    "cs.DL",
    "cs.DM",
    "cs.DS",
    "cs.ET",
    "cs.FL",
    "cs.GL",
    "cs.GR",
    "cs.GT",
    "cs.HC",
    "cs.IR",
    "cs.LG",
    "cs.LO",
    "cs.MA",
    "cs.MM",
    "cs.MS",
    "cs.NA",
    "cs.NE",
    "cs.NI",
    "cs.OH",
    "cs.OS",
    "cs.PF",
    "cs.PL",
    "cs.RO",
    "cs.SC",
    "cs.SD",
    "cs.SE",
    "cs.SI",
    "cs.SY"
]
pipeline.add_processor(Processors.FilterOnMetadataValue("categories", cs_values))

In [6]:
#MetadataFieldDiscarder
fields_to_discard = ["update_date", "versions", "comments", "report-no", "license", "id", "abstract", "categories", "submitter", "journal-ref", "authors_parsed"]
pipeline.add_processor(Processors.MetadataFieldDiscarder(fields_to_discard))

In [7]:
#TextKeywordCut
pipeline.add_processor(Processors.TextKeywordCut("introduction"))

In [8]:
#TextReplaceFilter
pipeline.add_processor(Processors.TextReplaceFilter(r"\{{(.*?)\}}", ""))

In [9]:
#HaystackPreProcessor
from haystack.preprocessor import PreProcessor
pre_processor = PreProcessor(
    clean_empty_lines=True,
    clean_whitespace=True,
    clean_header_footer=True,
    split_by="word",
    split_length=200,
    split_respect_sentence_boundary=False,
    split_overlap=0)
pipeline.add_processor(Processors.HaystackPreProcessor(pre_processor))

In [10]:
#TextfileSink
pipeline.add_sink(Sinks.TextfileSink("/home/daniel/storage/KD/testoutput", ["arixive-id", "_split_id"]))

In [11]:
print(pipeline)

(UnarxiveAdapter) ---> *MetadataArxiveEnricher* ---> *FilterOnMetadataValue* ---> *MetadataFieldDiscarder* ---> *TextKeywordCut* ---> *TextReplaceFilter* ---> *HaystackPreProcessor*
===> |_TextfileSink_|


### Use Pipeline to process documents

In [12]:
pipeline.process()

Processing batches: [############################################################] 10/10
